In [428]:
import os
import pandas as pd
import gradio as gr

import torch

from utils import utils

In [429]:
import warnings
warnings.filterwarnings("ignore")

In [430]:
model_path = os.path.join(".", 'models', 'binaries')
models = [model for model in os.listdir(model_path) if model.endswith(".pth")]
models.sort()

final_model = models[-1]
final_model_path = os.path.join(model_path, final_model)

del models

In [431]:
embedding_state = torch.load(final_model_path)

n_users = embedding_state['n_users']
n_movies = embedding_state['n_movies']
n_factors = embedding_state['n_factors']

movie_bias = embedding_state['state_dict']['movie_bias.weight']
movie_factors = embedding_state['state_dict']['movie_factors.weight']

userLabel_to_Id = embedding_state["userLabel_to_Id"]
movieLabel_to_Id = embedding_state["movieLabel_to_Id"]

del embedding_state, final_model_path

In [432]:
resource_path = os.path.join(".", 'data', 'ml-25m')
movies_csv = os.path.join(resource_path, 'movies.csv')
links_csv = os.path.join(resource_path, 'links.csv')

movies = pd.read_csv(movies_csv)
movies['genres'] = movies.genres.apply(lambda genres: genres.split('|'))

links = pd.read_csv(links_csv, dtype = {'movieId': int, 'imdbId': str, 'tmdbId': str})
movies = movies.merge(links)
del links

In [433]:
def get_movie_id(label):
    id = movieLabel_to_Id([label])
    return id.item(0)

movies_dict = {
    'label': [],
    'title': [],
    'genres':  [],
    'imdbId': []
}

for label in range(n_movies):
    movies_dict['label'].append(label)
    movies_dict['title'].append(movies[movies['movieId']==get_movie_id(label)]['title'].item())
    movies_dict['genres'].append(movies[movies['movieId']==get_movie_id(label)]['genres'].item())
    movies_dict['imdbId'].append(movies[movies['movieId']==get_movie_id(label)]['imdbId'].item())

movies = pd.DataFrame(movies_dict)
del movies_dict
movies

,label,title,genres,imdbId
0,0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",0114709
1,1,Jumanji (1995),"[Adventure, Children, Fantasy]",0113497
2,2,Heat (1995),"[Action, Crime, Thriller]",0113277
3,3,GoldenEye (1995),"[Action, Adventure, Thriller]",0113189
4,4,Ace Ventura: When Nature Calls (1995),[Comedy],0112281
...,...,...,...,...
195,195,Up (2009),"[Adventure, Animation, Children, Drama]",1049413
196,196,Avatar (2009),"[Action, Adventure, Sci-Fi, IMAX]",0499549
197,197,Inception (2010),"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",1375666
198,198,Django Unchained (2012),"[Action, Drama, Western]",1853728


In [434]:
def get_movie_name(label):
    return movies[movies['label']==label]['title'].item()

In [435]:
user_movies = dict()

In [436]:
def add_movie_to_list(movie_title, rating):
    label = movies[movies['title']==movie_title].label.item()
    
    user_movies[label] = rating
    print(user_movies)

In [437]:
def disp_user_movies():
    if not len(user_movies):
        return gr.Markdown('No movies added to list yet')
    
    md_table = 'No. | Movie | Rating\n---|---|---'
    sorted_user_movies = sorted(user_movies.items(), key=lambda x:x[1], reverse=True)
    for i, (label, rating) in enumerate(sorted_user_movies):
        md_table += f'\n{i+1} | {get_movie_name(label)} | {rating}'

    return gr.Markdown(md_table)

def disp_user_recommendations(bias_weight):
    if not len(user_movies):
        return gr.Markdown('No movies added to list yet')
    
    recommendations = dict()
    for label, rating in user_movies.items():
        recommendations = utils.get_recommendations(label, movie_factors, movie_bias, rating, bias_weight, recommendations)

    recommendations = sorted(recommendations.items(), key=lambda x:x[1], reverse=True)
    recommendations = [label for label, _ in recommendations][:20]

    md_table = 'No. | Movie\n---|---'
    for i, label in enumerate(recommendations):
        md_table += f'\n{i+1} | {get_movie_name(label)}'

    return gr.Markdown(md_table)


In [438]:
def set_bw(bias_wt):
    return disp_user_recommendations(bias_wt)

In [439]:
def add_movie(movie_title, rating, bias_wt):
    add_movie_to_list(movie_title, rating)

    return [disp_user_movies(), disp_user_recommendations(bias_wt)]

In [440]:
def clear_user_movies():
    user_movies.clear()

    return [disp_user_movies(), disp_user_recommendations(0.2)]

In [441]:
movie_list = sorted(movies.title.values.tolist())

In [442]:
with gr.Blocks() as app:
    gr.Markdown('<h1 style="text-align: center;">Movie Recommendation System</h1>')
    gr.Markdown('<p style="text-align: center;">Project by: Alex Prateek Shankar (2248302) and Sneh Shah (2248318)</p>')

    with gr.Blocks():
        movie_title = gr.Dropdown(
            choices=movie_list,
            label="Select Movie"
            )
        rating = gr.Slider(
            0.0, 5.0, 
            value=2.5, 
            step=0.1, 
            label="Rating", 
            info="Give your rating for the movie",
            interactive=True
            )
        add_btn = gr.Button(
            "Add Movie",
            label="Click here to add movies to your list"
            )
        
    gr.Markdown('<hr>')

    with gr.Blocks():
        gr.Markdown('<h2 style="text-align: center;">My Movies</h2>')

        my_movies = disp_user_movies()

        clear_movies = gr.Button(
            "Clear My Movies",
            label="Click here to clear your movies list"
        )

    gr.Markdown('<hr>')

    with gr.Blocks():
        gr.Markdown('<h2 style="text-align: center;">Bias Weight</h2>')

        bias_wt = gr.Slider(
            0.0, 1.0,
            value=0.2,
            step=0.1,
            label="Bias Weight",
            info="This will affect how the pre-existing movie ratings will affect your recommendations"
        )
        bw_btn = gr.Button(
            "Set Bias Weight",
            label="Click here to set the Bias Weight"
        )

    gr.Markdown('<hr>')

    with gr.Blocks():
        gr.Markdown('<h2 style="text-align: center;">My Top 20 Recommendations</h2>')

        my_recommendations = disp_user_recommendations(0.2)

    clear_movies.click(fn=clear_user_movies, outputs=[my_movies, my_recommendations])
    bw_btn.click(fn=set_bw, inputs=[bias_wt], outputs=[my_recommendations])
    add_btn.click(fn=add_movie, inputs=[movie_title, rating, bias_wt], outputs=[my_movies, my_recommendations])


app.title = "Neural Collaborative Filtering"
app.launch()

Running on local URL:  http://127.0.0.1:7893

To create a public link, set `share=True` in `launch()`.


{30: 4.8}
{30: 4.8, 181: 2}
{30: 4.8, 181: 2, 185: 4.3}
{185: 4.5}
